# Google Brain EDA

- 目的
  - Google Brainのデータ構成を確認する
  - 各特徴量パラメータを確認する

- 参考URL
  - コンペティションページ: https://www.kaggle.com/c/ventilator-pressure-prediction/overview
  - データページ: https://www.kaggle.com/c/ventilator-pressure-prediction/data?select=train.csv


## 0. インポート

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1. コンペティションの目的の確認

[原文]

The ventilator data used in this competition was produced using a modified open-source ventilator connected to an artificial bellows test lung via a respiratory circuit. The diagram below illustrates the setup, with the two control inputs highlighted in green and the state variable (airway pressure) to predict in blue. The first control input is a continuous variable from 0 to 100 representing the percentage the inspiratory solenoid valve is open to let air into the lung (i.e., 0 is completely closed and no air is let in and 100 is completely open). The second control input is a binary variable representing whether the exploratory valve is open (1) or closed (0) to let air out.

In this competition, participants are given numerous time series of breaths and will learn to predict the airway pressure in the respiratory circuit during the breath, given the time series of control inputs.

![](https://raw.githubusercontent.com/google/deluca-lung/main/assets/2020-10-02%20Ventilator%20diagram.svg)

Each time series represents an approximately 3-second breath. The files are organized such that each row is a time step in a breath and gives the two control signals, the resulting airway pressure, and relevant attributes of the lung, described below.

- (注釈)
  - ventilator...人工呼吸器
  - artificial bellows test lung...人工肺
  - bellows...ふいご
 
　　  ![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Bellows.png/120px-Bellows.png)
  
  - inspiratory, expiratory...吸気、in-は吸入される吸気、ex-は排出される吸気のことを示していると思われる
  
[概略]

人工呼吸器のテストを実施するために、人工肺(呼吸回路)に接続してテストを実施した.画像はその回路を示している.緑はコントロール値.青は状態値を示している.このコンペティションでは、多数の時系列の呼吸とそれに対する制御入力から呼吸中の呼吸回路の気道内圧を予測することを目的とする.

[パラメータについて]

- 各特徴量について
    - **id**: 全データに対する時刻ID
    - **breath_id**: 呼吸に対する時刻ID
    - **R**: 気道がどの程度制限されているかを示すパラメータ(cmH2O/L/S)  
      風船イメージ-ストローの直径を変えることでRを変更することができるが、Rが大きいほど空気を入れる時に必要な力は大きくなる.
    - **C**: 肺のコンプライアンス(追従性？)を示すパラメータ(mL/cmH2O)  
      風船イメージ-風船のゴムの厚さを変更することでCが変わる。Cが高いほど風船は薄くなり、空気を入れるときに必要な力は小さくなる.
    - **time_step**: 実際のタイムスタンプ.
    - **u_in**: 吸気電磁弁の制御入力.  
      肺に空気を入れるために吸気電磁弁が開いているパーセンテージを表す0~100までの連続数.
    - **u_out**: 探索的電磁弁の制御入力.  
      空気を輩出するために探索バルブが開いているかを表すバイナリ変数(1)閉(2)開.
      
      
- 目的変数について
    - **pressure**: 呼吸回路で測定された気道内圧=cmH2O.

## 2. データの確認

### 2.1. trainデータの確認

In [ ]:
train_data = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
train_data

### 2.2. testデータの確認

In [ ]:
test_data = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")
test_data

### 2.3. sample_submissionデータの確認

In [ ]:
sample_data = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
sample_data

## 3. データの可視化

### 3.1. breath_idごとにデータを分けてプロットを作成

breath_idごとに、C,Rの値が変わっており、pressureの値もリセットされている様子なので、breath_idごとにデータを分けて時系列的にデータをプロットしてみる.

**train_data**

In [ ]:
breath_id_area = 10
check_param_dict = {
    'R':[-1, 60],
    'C':[-1, 60],
    'time_step':[-1, 3],
    'u_in':[-1, 35],
    'u_out':[-0.1, 1.1],
    'pressure':[-1, 40]
}
fig = plt.figure(figsize=[50, 15])

for i_breath_id, breath_id in enumerate(train_data.loc[:, 'breath_id'].unique()[:breath_id_area]):
    breath_id_df = train_data[train_data.loc[:, 'breath_id']==breath_id]
    
    for i_param, (param, param_range) in enumerate(check_param_dict.items()):
        fig.add_subplot(len(check_param_dict), breath_id_area, i_breath_id+breath_id_area*i_param+1)
        if param is 'pressure':
            plt.plot(breath_id_df.loc[:, 'id'], breath_id_df.loc[:, param], c='r')
        else:
            plt.plot(breath_id_df.loc[:, 'id'], breath_id_df.loc[:, param], c='b')
        if i_param is 0: plt.title(f"breath_id={breath_id}")
        plt.ylim(param_range)
        plt.xlabel('id'); plt.ylabel(param)

plt.show()

**test_data**

In [ ]:
breath_id_area = 10
check_param_dict = {
    'R':[-1, 60],
    'C':[-1, 60],
    'time_step':[-1, 3],
    'u_in':[-1, 35],
    'u_out':[-0.1, 1.1],
}
fig = plt.figure(figsize=[50, 15])

for i_breath_id, breath_id in enumerate(test_data.loc[:, 'breath_id'].unique()[:breath_id_area]):
    breath_id_df = test_data[test_data.loc[:, 'breath_id']==breath_id]
    
    for i_param, (param, param_range) in enumerate(check_param_dict.items()):
        fig.add_subplot(len(check_param_dict), breath_id_area, i_breath_id+breath_id_area*i_param+1)
        if param is 'pressure':
            plt.plot(breath_id_df.loc[:, 'id'], breath_id_df.loc[:, param], c='r')
        else:
            plt.plot(breath_id_df.loc[:, 'id'], breath_id_df.loc[:, param], c='b')
        if i_param is 0: plt.title(f"breath_id={breath_id}")
        plt.ylim(param_range)
        plt.xlabel('id'); plt.ylabel(param)

plt.show()

[感想]
- u_out=0の時はu_inに追従してpressureが上昇しているのがわかる。また、u_outが1になるとpressureが著しく低下していることがわかる.
- C,Rの値に応じて、pressureの追従の仕方が変わっている.
- pressureは時系列に対して連続の値なので、学習させるときは前の時間のpressureの値、および上昇率を推論に使用できるとよさそう.
- trainとtestでidは重複していない.

### 3.2. time_stepの間隔を確認する

time_stepは主に0.03[sec]ごとに存在している様子だが、離散的な場合は時間ごとにu_inの変化量を重みづけしたりする必要があるため、ヒストグラムや最大値、最小値を確認してそのあたりを確認する.

- **train_data**

In [ ]:
train_time_diff = (train_data['time_step'] - train_data['time_step'].shift(1))
train_time_diff[train_data['breath_id']!=train_data['breath_id'].shift(1)] = np.average(time_diff)
plt.hist(train_time_diff, bins=30, range=[0, 0.3])
ax = ax = plt.gca()
ax.set_yscale('log')
plt.show()

- **test_data**

In [ ]:
test_time_diff = (test_data['time_step'] - test_data['time_step'].shift(1))
test_time_diff[test_data['breath_id']!=test_data['breath_id'].shift(1)] = np.average(time_diff)
plt.hist(test_time_diff, bins=30, range=[0, 0.3])
ax = ax = plt.gca()
ax.set_yscale('log')
plt.show()

- 0.25あたりに飛んでいるデータがあることがわかる
- 0,03台のデータが多いが、0.4台のデータも一部あることがわかる

→時系列データとして扱うときに、time_stepを考慮してデータを変換する必要があるように感じる

以上